In [ ]:
#Section 2
#َContent-Based Recommendation :

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

ratings = pd.read_csv("/content/sample_data/rating.csv")

metadata = pd.read_csv("/content/sample_data/anime.csv")


#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['name'])

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#Construct a reverse map of indices and movie titles
indices = pd.Series(metadata.index, index=metadata['anime_id']).drop_duplicates()
indices_na = pd.Series(metadata.index, index=metadata['name']).drop_duplicates()


#get top videos that user has watched
def get_movies(user_id):

  select = ratings.loc[ratings['user_id'] == user_id]
  sel = select[['anime_id', 'rating']]
  sel_sorted = sel.sort_values(by = 'rating', ascending = False)

  sel_mov = sel_sorted['anime_id'].tolist()

  all_mov = sel_sorted['anime_id'].tolist()

  m_top = sel_mov[0:10]

  return m_top , all_mov
  

N = len(metadata['name'])
	
name_dic = dict(zip(list(range(N)), (metadata["name"])))
id_dic = dict(zip(list(range(N)), (metadata["anime_id"])))


def get_recommendations(top_movies, cosine_sim=cosine_sim):

  rec_mov = []
  
  for i in top_movies:

    # Get the index of the movie that matches the title
    idx = indices[i]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the most similar movies
    sim_scores = sim_scores[1:3]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    for j in movie_indices:
      vid_id = id_dic[j]
      vid_name = name_dic[j]
      if (vid_name not in rec_mov) and (vid_id not in all_mov):
        rec_mov.append(vid_name)

 # Return the top most similar movies
  return rec_mov

print("")
print("for user id 3")
print("")

print("Suggested videos :")

top_movies , all_mov = get_movies(3)

re = get_recommendations(top_movies)[0:10]

print("")

for i in re:

 print(i)
 print("")
